In [1]:
%cd ..

/Users/erwan/Programmes/2022 Datacamp/solar_wind


### Tests d'une régression logistique et d'un histboost
- Transformation de la base de données : base, r_small_avg, l_small_avg, diff-mean, mean-ratio 
- Base d'entraînement et de tests modifiée.
- Pas de re-échantillonnage
- Pré-traitement du predict après entrainement (rolling 18h center=True, modifiable)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from problem import get_train_data, get_test_data
from problem import turn_prediction_to_event_list

# Fonction perso
from display import plot_event, multiple_plots, consecutive_plots, display_timeline, show_densities
from display import display_res


In [3]:
def get_fe_data(var_s):
    """
    Prend data_train['Beta'] au format brut (hors seuillabge ) et renvoie la série avec les traitements
    définis ci-dessous
    """

    # Moyenne mobile faible sur les valeurs pour espérer réduire dors et déjà le bruit 
    var_s = var_s.rolling('30 min', center=True).mean()

    # Différence entre moyenne longue et moyenne courte (mobiles) 
    # Met en avant les transitions entre états
    var_l_2h_s = var_s.rolling('2h').mean()
    var_r_2h_s = var_s.iloc[::-1].rolling('2h').mean().iloc[::-1]

    # Ratio entre une moyenne mobile courte et une très longue
    # Met en avant les valeurs faibles de Beta
    var_4h_s = var_s.rolling('4h', center=True).mean()
    var_long_s = var_s.rolling('4 d', center=True).mean()

    var_l_20h_s = var_s.rolling('20h').mean()
    var_r_20h_s = var_s.iloc[::-1].rolling('20h').mean().iloc[::-1]

    df = pd.DataFrame({})
    df['base-value'] = var_s

    df['l_small_avg'] = var_l_2h_s.copy()
    df['r_small_avg'] = var_r_2h_s.copy()
    df['diff-mean'] = (df['r_small_avg'] - df['l_small_avg']).abs()

    df['mean-ratio'] = var_4h_s / var_long_s

    # Pour les tests de moyennes mobiles longues
    # df['l_global_avg'] = var_l_20h_s.copy()
    # df['r_global_avg'] = var_r_20h_s.copy()

    return df

In [4]:
# Données
data_train, labels_train = get_train_data()
data_test, labels_test = get_test_data()

# Évenements
events = turn_prediction_to_event_list(labels_train)
no_events = turn_prediction_to_event_list(labels_train == 0)

In [5]:
seuil = 50
var_train_s = data_train['Beta'].copy()
var_train_s = var_train_s.map(lambda x: min(x, seuil))

var_test_s = data_test['Beta'].copy()
var_test_s = var_test_s.map(lambda x: min(x, seuil))

In [6]:
fe_train_data = get_fe_data(var_train_s)
fe_test_data = get_fe_data(var_test_s)

fe_train_data.head(3)

,base-value,l_small_avg,r_small_avg,diff-mean,mean-ratio
1997-10-01 00:00:00,8.395598,8.395598,9.201718,0.806121,14.461996
1997-10-01 00:10:00,9.571798,8.983698,8.543898,0.439800,13.486491
1997-10-01 00:20:00,12.369014,10.112137,7.784553,2.327584,12.637000


In [7]:
fe_test_data.head(3)

,base-value,l_small_avg,r_small_avg,diff-mean,mean-ratio
2008-01-01 00:00:00,0.903901,0.903901,0.806726,0.097175,0.402711
2008-01-01 00:10:00,0.895658,0.899779,0.795686,0.104094,0.404263
2008-01-01 00:20:00,0.839025,0.879528,0.788897,0.090631,0.413132


### Entraînement des modèles
- Comparaison fe_data et small_fe_data (sans les moyennes gauches et droites)

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier

In [9]:
log_model = LogisticRegression(
    penalty="l2",
    max_iter=1000,
    random_state=0,
    solver='lbfgs'
)

In [10]:
# ENTRAÎNEMENT SUR LA BASE DE DONNÉES AVEC R_SMALL ET L_SMALL
# PAS DE RÉ-ÉCHANTILLONAGE SUR LA BDD

log_model.fit(fe_train_data, labels_train)
print('Coefs', log_model.coef_)
display_res(fe_test_data, labels_test, smooth=False, models=[log_model])

Coefs [[ 1.25639518 -6.19699049 -6.11041614  5.82282206  0.15213744]]
Loss : 0.13417136204630736
              precision    recall  f1-score   support

           0       0.95      1.00      0.97    191755
           1       0.94      0.21      0.35     13819

    accuracy                           0.95    205574
   macro avg       0.94      0.61      0.66    205574
weighted avg       0.95      0.95      0.93    205574

ev_prec 0.3846153846153846
ev_rec 0.09345794392523366
-------------


- Les coefficients ne sont pas très important (en terme de norme), régulariser davantage ne semble pas vraiment avoir d'intérêt

In [11]:
# ENTRAÎNEMENT SUR LA BASE DE DONNÉES SANS R_SMALL ET L_SMALL
# PAS DE RÉ-ÉCHANTILLONAGE SUR LA BDD

drop_cols = ['r_small_avg', 'l_small_avg']
print('Colonnes retirées :', drop_cols)
print('Colonnes restantes :', fe_train_data.drop(columns=drop_cols).columns.values)
log_model.fit(fe_train_data.drop(columns=drop_cols), labels_train)
print('Coefs', log_model.coef_)
display_res(fe_test_data.drop(columns=drop_cols), labels_test, smooth=False, models=[log_model])

Colonnes retirées : ['r_small_avg', 'l_small_avg']
Colonnes restantes : ['base-value' 'diff-mean' 'mean-ratio']
Coefs [[-5.80860613  0.49850662 -1.03068782]]
Loss : 0.15294442311281467
              precision    recall  f1-score   support

           0       0.94      1.00      0.97    191755
           1       1.00      0.10      0.17     13819

    accuracy                           0.94    205574
   macro avg       0.97      0.55      0.57    205574
weighted avg       0.94      0.94      0.92    205574

ev_prec 0.25
ev_rec 0.028037383177570097
-------------


- Comparaison avec HistBoost :

In [12]:
histboost_model = HistGradientBoostingClassifier(
    learning_rate=0.1,
    max_iter=1000,
    random_state=2
)

In [13]:
# ENTRAÎNEMENT SUR LA BASE DE DONNÉES AVEC R_SMALL ET L_SMALL
# PAS DE RÉ-ÉCHANTILLONAGE SUR LA BDD

histboost_model.fit(fe_train_data, labels_train)
display_res(fe_test_data, labels_test, [histboost_model])

In [14]:
# ENTRAÎNEMENT SUR LA BASE DE DONNÉES SANS R_SMALL ET L_SMALL
# PAS DE RÉ-ÉCHANTILLONAGE SUR LA BDD

drop_cols = ['r_small_avg', 'l_small_avg']
print('Colonnes retirées :', drop_cols)
print('Colonnes restantes :', fe_train_data.drop(columns=drop_cols).columns)
histboost_model.fit(fe_train_data.drop(columns=drop_cols), labels_train)
display_res(fe_test_data.drop(columns=drop_cols), labels_test, [histboost_model])

Colonnes retirées : ['r_small_avg', 'l_small_avg']
Colonnes restantes : Index(['base-value', 'diff-mean', 'mean-ratio'], dtype='object')
